In [ ]:
# Feed forward NN
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import pandas as pd
from sklearn import preprocessing

In [ ]:
df = pd.read_csv('/content/heart_failure_clinical_records_dataset.csv')

In [ ]:
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
class Dataset(Dataset):
    def __init__(self, file):
        self.data = pd.read_csv(file)
        input_features = self.data.values[:,0:11].astype('float32')
        self.x_train = torch.from_numpy(input_features)

        target_label = self.data['DEATH_EVENT'].values
        self.y_train = torch.from_numpy(target_label)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.x_train[idx],self.y_train[idx])

In [ ]:
dataset = Dataset('/content/heart_failure_clinical_records_dataset.csv')
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [ ]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, output_dim)


    def forward(self, x):
        x = F.softmax(self.layer1(x))
        return x

In [ ]:
model = Model(11, 2)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

Train

In [ ]:
epochs = 400

for epoch in range(1,epochs+1):
    print("EPOCH:",epoch,end=" ")
    running_loss=0
    running_acc=0

    for data,labels in dataloader:
        data,labels=data.to(device),labels.to(device)
        optimizer.zero_grad()
        output=model.forward(data)
        loss=criterion(output,labels)

        result=torch.argmax(output,dim=1)
        running_loss+=loss.item()
        running_acc+=torch.mean((result==labels).type(torch.float))

        loss.backward()
        optimizer.step()
    else:
        train_loss=running_loss/len(dataloader)
        train_acc=running_acc/len(dataloader)

        print("Training Loss: {:.3f}".format(train_loss),end=" ")

        print("Train Accuracy: {:.2f}%".format(train_acc.item()*100))

EPOCH: 1 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH: 2 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH: 3 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 4 Training Loss: 0.638 Train Accuracy: 67.52%
EPOCH: 5 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 6 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 7 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 8 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 9 Training Loss: 0.629 Train Accuracy: 68.42%
EPOCH: 10 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 11 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 12 Training Loss: 0.634 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


Train Accuracy: 67.97%
EPOCH: 13 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH: 14 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 15 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 16 Training Loss: 0.632 Train Accuracy: 68.12%
EPOCH: 17 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH: 18 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 19 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 20 Training Loss: 0.632 Train Accuracy: 68.12%
EPOCH: 21 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 22 Training Loss: 0.634 Train Accuracy: 67.97%
EPOCH: 23 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH: 24 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 25 Training Loss: 0.629 Train Accuracy: 68.42%
EPOCH: 26 Training Loss: 0.638 Train Accuracy: 67.52%
EPOCH: 27 Training Loss: 0.638 Train Accuracy: 67.52%
EPOCH: 28 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 29 Training Loss: 0.635 Train Accuracy: 67.82%
EPOCH: 30 Training Loss: 0.637 Train Accuracy: 67.67%
EPOCH

Train

In [ ]:
test = torch.tensor([[49.0,	1,	80,	0,	30,	1,	427000.0,	1.0,	138,	0,	0]])

In [ ]:
test.shape

torch.Size([1, 11])

In [ ]:
model(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


tensor([[1., 0.]], grad_fn=<SoftmaxBackward0>)

In [ ]:
model.layer1.weight

Parameter containing:
tensor([[ 0.2216, -0.0249, -0.1352,  0.0252,  0.2193, -0.1790,  0.2713,  0.0413,
         -0.1346, -0.1240,  0.0103],
        [-0.0094,  0.0311,  0.0688, -0.2302, -0.2204, -0.2733, -0.1166, -0.1306,
          0.1314, -0.1471, -0.2015]], requires_grad=True)